# <center>Change Instrument used in a MIDI</center>

This notebook will read in a MIDI file, print the key it is in and output a new MIDI file with the notes detected being played using a selected instrument.

This notebook was created as a separate app implementing interesting functionality that was discovered during the recreation of songs.

## Directions for Use

1. Place the desired MIDI file(s) in the folder /Converter/input
2. Run the first two code snippets to read in the file(s)
3. Choose the desired output instrument using the third code snippet:
    * Piano
    * Bagpipes
    * Flute
    * Clarinet
    * Ocarina
    * Harmonica
    * Steel Drum
    * Vocals
    * Soprano (vocals)
    * Bass (vocals)
    * Guitar
    * Glec_guitar
    * Violin
    * Saxophone
    * Trombone
    * Trumpet
    * Enlish Horn
4. Run the final code snippet to convert the instrument
5. Play the output MIDI file located in the folder /Converter/output

In [20]:
import glob
from music21 import converter, instrument, note, chord, stream, duration

In [13]:
def get_notes(path):
    """
        Gets all notes and chords from midi file
    """
    notes = []
    names = []

    for file in glob.glob(path + "*.mid"):        
        song = []
        midi = converter.parse(file)
        
        # Get some info about the file
        key = midi.analyze('key')
        key_string = key.tonic.name + ' ' + key.mode
        
        print("Parsing %s" % file)
        print(key_string)
        
        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
            instrumCount = 'instrument' if len(s2.parts) == 1 else 'instruments'
            print(f'This song has {len(s2.parts)} {instrumCount} in use:')
            for part in s2.parts:
                print(f'\t {part.getInstrument()}')
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                song.append([str(element.pitch), element.offset, element.duration])
            elif isinstance(element, chord.Chord):
                song_note = '.'.join(str(n) for n in element.normalOrder)
                song.append([song_note, element.offset, element.duration])
        notes.append(song)
        names.append(file.strip('../Converter/input\\'))

    return notes, names

input_path = "../Converter/input/"
notes, names = get_notes(input_path)

Parsing ../Converter/input\LSTM_output_piano_1.mid
D major
This song has 1 instrument in use:
	 Piano


Now that the file has been read in, select which instrument to use.

In [18]:
# Select instrument
instruments = {
    'piano': instrument.Piano(),
    'bagpipes': instrument.Bagpipes(),
    'flute': instrument.Flute(),
    'clarinet': instrument.Clarinet(),
    'ocarina': instrument.Ocarina(),
    'harmonica': instrument.Harmonica(),
    'steel_drum': instrument.SteelDrum(),
    'vocals': instrument.Vocalist(),
    'soprano': instrument.Soprano(),
    'bass': instrument.Bass(),
    'guitar': instrument.Guitar(),
    'elec_guitar': instrument.ElectricGuitar(),
    'violin': instrument.Violin(),
    'saxophone': instrument.Saxophone(),
    'trombone': instrument.Trombone(),
    'trumpet': instrument.Trumpet(),
    'english_horn': instrument.EnglishHorn(),
}
instrument_string = 'ocarina'
instrument_choice = instruments[instrument_string]

Finally, the track can be reconstructed using the new instrument.

In [19]:
from music21 import duration as D

def create_midi(prediction_output, filename, instrument_choice):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    output_notes = []
    output_notes.append(instrument_choice)

    # create note and chord objects based on the values generated by the model
    count = 0
    for pattern in prediction_output:
        note_str = pattern[0]
        offset_str = pattern[1]
        duration_str = pattern[2]
        if "#-" in note_str:# To fix a rare exception using 2 accidentals
            continue
        # pattern is a chord
        if ('.' in note_str) or note_str.isdigit():
            notes_in_chord = note_str.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset_str
            new_note.duration = duration_str
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(note_str)
            new_note.offset = offset_str
            new_note.duration = duration_str
            output_notes.append(new_note)
        count += 1

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}'.format(filename))
    
for (song_notes, song_names) in zip(notes, names):
    create_midi(song_notes, '../converter/output/CONVERTED_' + instrument_string + '_' + song_names, instrument_choice)